In [1]:
#Do the migration
import sqlalchemy
from pgvector.sqlalchemy import Vector
from sqlalchemy import create_engine, insert, select, text, Integer
from sqlalchemy.orm import declarative_base, mapped_column, Session
from flask import Flask, session
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
import time

from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.pgvector import PGVector
from langchain_text_splitters import CharacterTextSplitter

In [2]:
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_DB = os.getenv('POSTGRES_DB')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')

sqlversion = sqlalchemy.__version__

CONNECTION_STRING = f'postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}'

print(sqlversion)

2.0.28


In [3]:
##################### EXAMPLE EMBEDDINGS ############################
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
text = "This is a test document."
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text, "This is not a test document."])

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
loader = TextLoader("DB/RawBCLaws/all_act_titles.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator="\n")
docs = text_splitter.split_documents(documents)

Created a chunk of size 31, which is longer than the specified 1
Created a chunk of size 31, which is longer than the specified 1
Created a chunk of size 28, which is longer than the specified 1
Created a chunk of size 12, which is longer than the specified 1
Created a chunk of size 22, which is longer than the specified 1
Created a chunk of size 22, which is longer than the specified 1
Created a chunk of size 37, which is longer than the specified 1
Created a chunk of size 19, which is longer than the specified 1
Created a chunk of size 32, which is longer than the specified 1
Created a chunk of size 11, which is longer than the specified 1
Created a chunk of size 17, which is longer than the specified 1
Created a chunk of size 20, which is longer than the specified 1
Created a chunk of size 11, which is longer than the specified 1
Created a chunk of size 26, which is longer than the specified 1
Created a chunk of size 57, which is longer than the specified 1
Created a chunk of size 1

In [6]:
COLLECTION_NAME = "bc_law_titles"

# If the database table already exists, delete it
db = PGVector.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=True,
)


In [7]:
query = "Is there a law on tenancy act?"
print('\n' + query + '\n')
docs_with_score = db.similarity_search_with_score(query)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)


Is there a law on tenancy act?

--------------------------------------------------------------------------------
Score:  0.19183902789121554
Residential Tenancy Act
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.22385208464550332
Commercial Tenancy Act
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.3256534143892864
Manufactured Home Park Tenancy Act
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.35948024037598014
Property Law Act
--------------------------------------------------------------------------------


In [8]:
store = PGVector(
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding_function=embeddings,
) 

query = "Is there any laws for landlords?"
print('\n' + query + '\n')
docs_with_score = store.similarity_search_with_score(query)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)



Is there any laws for landlords?

--------------------------------------------------------------------------------
Score:  0.44390225410461426
Residential Tenancy Act
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.4451255136877871
Short-Term Rental Accommodations Act
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.4734227657318115
Rent Distress Act
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.4775239825248718
Commercial Tenancy Act
--------------------------------------------------------------------------------
